In [ ]:
import sys
sys.path.append('./src')

import io_utils

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from metrics import unique_products, category_revenue
from metrics import units_per_city, top_cities, category_per_city
from metrics import revenue_by_month, revenue_by_day_of_month
from metrics import detect_anomalies

import viz

df = io_utils.get_df()


### 1. Vad säljer vi? – vilka kategorier driver mest intäkt?
- Kategorier 'Electronics', 'Sports' respektive ' Clothing' driver mest intäkt.

In [ ]:
print(f"Vi säljer följande unika kategorier: \n {unique_products}")

print(f"\nIntäkt per kategori: \n {category_revenue}")

In [ ]:
fig, ax =plt.subplots()
category_revenue.plot(kind="bar", ax=ax)
ax.set_title("Intäkt per kategori")
ax.set_xlabel("Kategori")
ax.set_ylabel("Intäkt")
ax.grid(True, axis='y')
plt.tight_layout()
plt.show()

### 2. Var säljer vi? – vilka städer står för störst intäkt?
- Stockholm, Göteborg respektive Malmö står för störst intakt.

In [ ]:
top_cities # sum of revenue per city

In [ ]:
units_per_city #sum of units per category and city

In [ ]:
category_per_city #sum of revenue per category and city.

In [ ]:
fig, ax = plt.subplots()
category_per_city.plot(kind='bar', ax=ax)
ax.set_title("Intäkter per stad och kategori")
ax.set_xlabel("Stad")
ax.set_ylabel("Intäkt")
ax.grid(True, axis='y')
plt.tight_layout()
plt.show()

### 3. När säljer vi? – finns tidsmönster/säsong i försäljningen?
- Vi säljer mest i januari och maj.

In [ ]:
revenue_by_month # Shows the total revenue for the different months


In [ ]:
revenue_by_day_of_month # Shows the best selling days of the months

In [ ]:
plt.plot(revenue_by_month.index, revenue_by_month.values)
plt.xlabel("Månad")
plt.ylabel("Intäkt")
plt.title("Intäkt per månad")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
plt.bar(revenue_by_day_of_month.index, revenue_by_day_of_month.values)
plt.xlabel("Dag")
plt.ylabel("Intäkt")
plt.title("Månadens bäst säljande dagar")
plt.grid(True)
plt.tight_layout()
plt.show()

### 4. Hur ser en typisk order ut? – AOV (Average Order Value) och spridning.
- Genomsnittligt ordervärde: 1401 kr.

In [ ]:

# AOV, standard deviation, minimum and maximum order value
order_value = df["revenue"].agg(["mean", "std", "min", "max"])
order_value.index = [
        "Genomsnittligt ordervärde:",
        "Standardavvikelse:",
        "Lägsta ordervärdet:",
        "Högsta ordervärdet:"
    ]
for name, value in order_value.astype(int).items():
  print(f"{name} {value} kr")


#Distribution of order values
fig, (ax_hist, ax_box) = plt.subplots(2, 1, figsize=(10, 6), gridspec_kw={"height_ratios":[4,1]})

ax_hist.hist(df["revenue"], bins=50)
ax_hist.set_title("Spridning av ordervärden")

ax_hist.set_ylabel("Antal ordrar")
ax_hist.grid(True, axis="y")

ax_box.boxplot(df["revenue"], vert=False)
ax_box.set_xlabel("Ordervärde (kr)")
ax_box.set_yticklabels("")
ax_box.grid(True, axis="x")

plt.tight_layout()



### 5. Topp-listor – topp-3 kategorier efter intäkt.

- **Topp-kategorier:** 1. Electronics, 2. Sports, 3. Clothing.

In [ ]:
# Summera intäkt per kategori
kategori_sum = df.groupby("category")["revenue"].sum()

# Plocka fram topp 3
top3 = kategori_sum.nlargest(3)

# Totala intäkten (avrundad uppåt till närmaste 500, med min 1000)
total = kategori_sum.sum()
total_rounded = max(1000, int(np.ceil(total / 500.0) * 500))

print("Top 3 kategorier baserat på omsättning:")
print(top3)
print(f"\nTotal omsättning: {total} (avrundad till {total_rounded})")

kategori_sum = df.groupby("category")["revenue"].sum()
top3 = kategori_sum.nlargest(3)
top3.plot(kind="bar", color="skyblue")

plt.title("Top 3 kategorier baserat på omsättning")
plt.ylabel("Total omsättning")

# Visa hela tal istället för 1e6
plt.gca().yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

plt.show()

### 6. Eventuella avvikelser – något oväntat mönster som sticker ut?


In [ ]:
df["date"] = pd.to_datetime(df["date"], errors="coerce")

daily_revenue = (
    df.set_index("date")
      .sort_index()["revenue"]
      .resample("D")
      .sum()
)

print("Antal dagar som analyseras:", daily_revenue.shape[0])

In [ ]:
Z = 3.0
anomalies, z = detect_anomalies(daily_revenue, threshold=Z)

In [ ]:
viz.plot_anomalies(daily_revenue, anomalies, z, Z)

Figur 1 visar daglig intäkt, där röda punkter markerar dagar som avviker kraftigt från det normala.  
Figur 2 visar motsvarande Z-poäng, där de röda strecken markerar gränsen för +/-3 standardavvikelser.

### Slutsatser och rekommendationer:

- **Topp-kategorier:** 1. Electronics, 2. Sports, 3. Clothing
- **Topp-städer:** 1. Stockholm, 2. Göteborg, 3. Malmö
- **Säsongstopp:** Januari

- Satsa på kategori 'Electronics' i Stockholm under januari.
- Undersök varför vi säljer sämre i mars.
- Undersök varför kategorierna 'Toys' och 'Beauty' säljer sämre.